# 1. Extract data from PDF files

### 1.1. Install and import libraries

In [1]:
# Install Libraries
%pip install pdfplumber pandas

  Obtaining dependency information for pdfplumber from https://files.pythonhosted.org/packages/12/28/3958ed81a9be317610ab73df32f1968076751d651c84dff1bcb45b7c6c0e/pdfplumber-0.11.8-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Obtaining dependency information for pdfminer.six==20251107 from https://files.pythonhosted.org/packages/64/29/d1d9f6b900191288b77613ddefb73ed35b48fb35e44aaf8b01b0422b759d/pdfminer_six-20251107-py3-none-any.whl.metadata
  Obtaining dependency information for pypdfium2>=4.18.0 from https://files.pythonhosted.org/packages/13/bf/4259b23a88b92bec8199e1a08a0821dbfbb465629c203bdbc49e2f993940/pypdfium2-5.0.0-py3-none-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 5.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import Libraries
import pdfplumber
import pandas as pd
import re

### 1.2. Helper function to load PDF and extract text

In [3]:
def extract_text_from_pdf(pdf_path):
  """
  This function reads a PDF file and returns one long string of text
  """
  text = ""
  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      page_text = page.extract_text()
      if page_text:
        text += page_text + "\n"
  return text

### 1.3. Extract text from files

In [4]:
fedex_pdf_1 = "files/FedEX 1.pdf"
fedex_pdf_2 = "files/FedEX 2.pdf"
evri_pdf = "files/Evri 1.pdf"

fedex_text_1 = extract_text_from_pdf(fedex_pdf_1)
fedex_text_2 = extract_text_from_pdf(fedex_pdf_2)
evri_text = extract_text_from_pdf(evri_pdf)

In [5]:
# print(fedex_text_1)
print(fedex_text_2)
# print(evri_text)

FedEx Express UK Transportation Limited Contact:fedex.com/en-gb/invoice Bank Account Number: 73895408
Express House VAT: GB354715057 BIC/SWIFT Code: MIDLGB22
Holly Lane FedEx Listed Agents No: 1628530 Bank Name: HSBC Bank Plc
Atherstone
IBAN: GB53MIDL40312473895408
CV9 2RY
United Kingdom Branch/Sort Code: 40-31-24
Online: https://www.fedex.com/payment/
Page 1 of 31
Customer Number: *****4754
Payment QR: Invoice Number: 817009883
Invoice Date: 15/10/2025
Due Date: 14/11/2025
C-RETAIL LTD Amount Due 1,345.50 GBP
THE DUKE, WELLINGTON ROAD
C&C
BURTON ON TRENT
DE14 2AB Customer VAT Nbr: GB974823775
UNITED KINGDOM
Freight Invoice
Shipment Ship Date Service Pieces Weight Reference Taxable Non Taxable Total
393933215765 06/10/2025 FedEx Priority 1 17.90 kg 2.99 0.00 2.99
We calculated your charges based on a dimensional weight of 17.9 kgs. The IATA standard was used to determine dimensional weight.
Fuel Surcharge - FedEx has applied a fuel surcharge of 17.25 % to this shipment.
Shipment Ship D

### 1.4. Parse FedEx lines

In [6]:
def parse_fedex(text):
  """
  This function extracts shipment lines from FedEx PDFs.
    - Look at the PDF text line by line.
    - Find lines that look like a shipment row:
        <shipment_number> <date> FedEx Priority ...
    - Grab the shipment number, date and the last numeric value on the line
      and treat that last number as the Total charge for the shipment.

  Regex pattern explanation:
    - ^(\d{9,}) matches a long shipment number at the start of the line.
    - (\d{2}/\d{2}/\d{4}) captures dates like 13/10/2025.
    - \d+\.\d+ finds decimal values such as 2.99, 17.10 etc.
        The last decimal value on the line is treated as the Total charge.
  """
  rows = []

  # Split PDF text into individual lines for easier processing

  for line in text.splitlines():
    # Match a shipment line: Starts with shipment_number + date
    pattern = r"^(\d{9,})\s+(\d{2}/\d{2}/\d{4})"
    m = re.match(pattern, line)

    if not m:
      # If line does not match shipment format, skip it
      continue

    # Extract shipment number and date
    shipment_number = m.group(1)
    shipment_date = m.group(2)

    # Extract all decimal numbers on the line
    nums = re.findall(r"\d+\.\d+", line)

    if not nums:
      # Skip if no numeric values found
      continue

    # The last decimal number is the Total charge for that shipment
    charge = float(nums[-1])

    rows.append({
        "shipment_number": shipment_number,
        "shipment_date": shipment_date,
        "charge": charge,
        "raw_line": line  # optional for debugging
    })
  return pd.DataFrame(rows)
    

### 1.5. Run parsing for both FedEx PDFs and combine

In [7]:
fedex_df1 = parse_fedex(fedex_text_1)
fedex_df2 = parse_fedex(fedex_text_2)

# Combine both invoices into one dataframe
fedex_df = pd.concat([fedex_df1, fedex_df2], ignore_index=True)

#### 1.5.1. FedEx data check


In [8]:
# Basic shape and first rows
print("Number of FedEx rows:", len(fedex_df))
fedex_df.head()

# Check for missing values
print("\nMissing values per column:")
print(fedex_df.isna().sum())

# Check if any charge is zero or negative
fedex_anomalies_basic = fedex_df[fedex_df["charge"] <= 0]
fedex_anomalies_basic


FedEx rows: 803

Missing values per column:
shipment_number    0
shipment_date      0
charge             0
raw_line           0
dtype: int64


,shipment_number,shipment_date,charge,raw_line


In [9]:
fedex_df1['charge'].unique()

array([ 2.99,  5.98, 19.35,  8.97])

In [11]:
fedex_df["charge"].describe()

count    803.000000
mean       3.082877
std        0.914972
min        2.990000
25%        2.990000
50%        2.990000
75%        2.990000
max       19.350000
Name: charge, dtype: float64

In [10]:
fedex_df1

,shipment_number,shipment_date,charge,raw_line
0,394183868992,13/10/2025,2.99,394183868992 13/10/2025 FedEx Priority 1 17.10...
1,394183887862,13/10/2025,2.99,394183887862 13/10/2025 FedEx Priority 1 17.60...
2,394183900668,13/10/2025,2.99,394183900668 13/10/2025 FedEx Priority 1 17.40...
3,394183910246,13/10/2025,2.99,394183910246 13/10/2025 FedEx Priority 1 18.30...
4,394183923474,13/10/2025,2.99,394183923474 13/10/2025 FedEx Priority 1 18.50...
...,...,...,...,...
425,394372710157,17/10/2025,2.99,394372710157 17/10/2025 FedEx Priority 1 1.00 ...
426,394373308575,17/10/2025,2.99,394373308575 17/10/2025 FedEx Priority 1 7.40 ...
427,394373314412,17/10/2025,2.99,394373314412 17/10/2025 FedEx Priority 1 15.00...
428,394374060659,17/10/2025,2.99,394374060659 17/10/2025 FedEx Priority 1 10.40...


### 1.6. Parse Evri invoice

In [24]:
def parse_evri(text):
    """
    This function extracts despatch service lines from Evri PDFs.

    Logic:
      - Read the PDF text line by line.
      - Identify lines that follow the Evri numeric pattern:
            <service text> <quantity> <unit_price> <VAT_code> <line_value>
        Example:
            Scottish Highlands & Islands Parcel 36 5.28 S 190.08
      - Extract:
          * service name (all text before the quantity column)
          * quantity (number of despatches)
          * unit price
          * line value

    Regex pattern explanation:
      - ^\\s* matches any leading spaces at the start of the line.
      - (.+?) captures the full service name as any characters, non-greedily,
        stopping just before the quantity column.
      - ([\\d,]+) captures the quantity column which may contain commas, for example 12,287.
      - (\\d+\\.\\d+) captures the unit price as a decimal number, for example 5.28.
      - [A-Z] matches the VAT code column, for example S or O.
      - ([\\d,]+\\.\\d+) captures the line total value, for example 190.08 or 25,679.83.

    """

    rows = []

    # Compile the pattern once for efficiency and readability
    pattern = r"^\s*(.+?)\s+([\d,]+)\s+(\d+\.\d+)\s+[A-Z]\s+([\d,]+\.\d+)"

    for line in text.splitlines():
        match = re.match(pattern, line)
        if not match:
            # Skip any line that does not match the expected Evri numeric structure
            continue

        service = match.group(1).strip()
        quantity = int(match.group(2).replace(",", ""))
        price = float(match.group(3))
        value = float(match.group(4).replace(",", ""))

        rows.append({
            "service": service,
            "quantity": quantity,
            "price": price,
            "value": value,
            "raw_line": line  # keep the raw line for checking or debugging
        })

    return pd.DataFrame(rows)


### 1.7. Run Evri Parser

In [25]:
evri_df = parse_evri(evri_text)

#### 1.7.1. Evri data check

In [26]:
print("Evri rows:", len(evri_df))
evri_df.head()

print("\nMissing values per column:")
print(evri_df.isna().sum())

# Zero or negative quantities or values
evri_anomalies_basic = evri_df[
    (evri_df["quantity"] <= 0) | 
    (evri_df["value"] <= 0)
]
evri_anomalies_basic


Evri rows: 32

Missing values per column:
service     0
quantity    0
price       0
value       0
raw_line    0
dtype: int64


,service,quantity,price,value,raw_line
0,WK35 Std Inv SUPERGROUP INTERNET LIMITED,1,0.0,0.0,WK35 Std Inv SUPERGROUP INTERNET LIMITED 1 0.0...
1,Charges between 26/10/25 to 01/11/25,1,0.0,0.0,Charges between 26/10/25 to 01/11/25 1 0.00 S ...
2,Password for backup & summary: Uew4hw5z,1,0.0,0.0,Password for backup & summary: Uew4hw5z 1 0.00...


In [27]:
evri_df.head()

,service,quantity,price,value,raw_line
0,WK35 Std Inv SUPERGROUP INTERNET LIMITED,1,0.00,0.00,WK35 Std Inv SUPERGROUP INTERNET LIMITED 1 0.0...
1,Charges between 26/10/25 to 01/11/25,1,0.00,0.00,Charges between 26/10/25 to 01/11/25 1 0.00 S ...
2,Password for backup & summary: Uew4hw5z,1,0.00,0.00,Password for backup & summary: Uew4hw5z 1 0.00...
3,Courier Parcel Despatch A (1501g to,2540,2.09,5308.60,"Courier Parcel Despatch A (1501g to 2,540 2.09..."
4,Courier Packet Despatch A (1g to 1500g),12287,2.09,25679.83,"Courier Packet Despatch A (1g to 1500g) 12,287..."


In [28]:
# Keep only rows where the Evri line_value is positive (real charges)
evri_core = evri_df[evri_df["value"] > 0].copy()

# Excluded rows (headers, meta rows, zero-value lines)
evri_excluded = evri_df[evri_df["value"] == 0].copy()

evri_core.head()
evri_excluded.head()


,service,quantity,price,value,raw_line
0,WK35 Std Inv SUPERGROUP INTERNET LIMITED,1,0.0,0.0,WK35 Std Inv SUPERGROUP INTERNET LIMITED 1 0.0...
1,Charges between 26/10/25 to 01/11/25,1,0.0,0.0,Charges between 26/10/25 to 01/11/25 1 0.00 S ...
2,Password for backup & summary: Uew4hw5z,1,0.0,0.0,Password for backup & summary: Uew4hw5z 1 0.00...


In [29]:
len(evri_core)

29

### 1.8. Export to CSV

In [30]:
fedex_df.to_csv("Extracted/fedex_extracted.csv", index=False)
evri_df.to_csv("Extracted/evri_extracted.csv", index=False)

print("CSV files saved:")
print("fedex_extracted.csv")
print("evri_extracted.csv")

CSV files saved:
fedex_extracted.csv
evri_extracted.csv


### 1.9. Quick validation checks

In [31]:
print("FedEx rows:", len(fedex_df))
print("FedEx total spend:", fedex_df["charge"].sum())

print("Evri rows:", len(evri_df))
print("Evri total despatches:", evri_df["quantity"].sum())
print("Evri total spend:", evri_df["value"].sum())

FedEx rows: 803
FedEx total spend: 2475.5500000000006
Evri rows: 32
Evri total despatches: 54746
Evri total spend: 43591.130000000005


# 2. Compare actual costs and fixed costs

### 2.1. The fixed cost rates

In [33]:
fixed_rate_fedex = 3.10
fixed_rate_evri = 2.44

### 2.2. FedEx calculations

In [34]:
# FedEx dataframe structure
# Each row is one despatch
# Column 'charge' is the charge for the despatch

# Total number of FedEx despatches
fedex_despatches = len(fedex_df)

# Total FedEx spend (sum of shipment charges)
fedex_spend = round(fedex_df["charge"].sum(), 3)

# Actual average cost per despatch
fedex_actual_avg = round(fedex_spend / fedex_despatches, 3) if fedex_despatches > 0 else 0

# Variance from fixed rate
fedex_variance = round(fedex_actual_avg - fixed_rate_fedex, 3)

# Label as over, under, or on rate
if fedex_variance > 0:
    fedex_status = "Over the fixed rate"
elif fedex_variance < 0:
    fedex_status = "Under the fixed rate"
else:
    fedex_status = "On the fixed rate"

fedex_despatches, fedex_spend, fedex_actual_avg, fedex_variance, fedex_status


(803, 2475.55, 3.083, -0.017, 'Under the fixed rate')

### 2.3. Evri calculations

In [35]:
# Evri dataframe structure (cleaned)
# evri_core contains only positive value charge lines

# Total number of Evri despatches
evri_despatches = evri_core["quantity"].sum()

# Total Evri spend (sum of line values)
evri_spend = round(evri_core["value"].sum(), 3)

# Actual average cost per despatch
evri_actual_avg = round(evri_spend / evri_despatches, 3) if evri_despatches > 0 else 0

# Variance from fixed rate
evri_variance = round(evri_actual_avg - fixed_rate_evri, 3)

# Label as over, under, or on rate
if evri_variance > 0:
    evri_status = "Over the fixed rate"
elif evri_variance < 0:
    evri_status = "Under the fixed rate"
else:
    evri_status = "On the fixed rate"

evri_despatches, evri_spend, evri_actual_avg, evri_variance, evri_status


(54743, 43591.13, 0.796, -1.644, 'Under the fixed rate')

# Summary

In [36]:
summary = pd.DataFrame([
    {
        "carrier": "FedEx",
        "despatches": fedex_despatches,
        "spend": fedex_spend,
        "avg_cost_per_despatch": fedex_actual_avg,
        "fixed_rate": fixed_rate_fedex,
        "variance": fedex_variance,
        "status": fedex_status
    },
    {
        "carrier": "Evri",
        "despatches": evri_despatches,
        "spend": evri_spend,
        "avg_cost_per_despatch": evri_actual_avg,
        "fixed_rate": fixed_rate_evri,
        "variance": evri_variance,
        "status": evri_status
    }
])

summary


,carrier,despatches,spend,avg_cost_per_despatch,fixed_rate,variance,status
0,FedEx,803,2475.55,3.083,3.10,-0.017,Under the fixed rate
1,Evri,54743,43591.13,0.796,2.44,-1.644,Under the fixed rate


In [37]:
# Main comparison summary
summary.to_csv("summary_for_dashboard.csv", index=False)

# Clean line level data for more detailed charts
fedex_df.to_csv("fedex_cleaned.csv", index=False)
evri_core.to_csv("evri_cleaned.csv", index=False)

# Anomalies and excluded lines for an exceptions view
evri_excluded.to_csv("evri_excluded.csv", index=False)

print("Files created:")
print("summary_for_dashboard.csv")
print("fedex_cleaned.csv")
print("evri_cleaned.csv")
print("evri_excluded.csv")


Files created:
summary_for_dashboard.csv
fedex_cleaned.csv
evri_cleaned.csv
evri_excluded.csv
